# W205 Project 3 - Data Pipeline and Analytics Report
### Eric Liu, Ian Dela Cruz, Luke Schanne

## Data Pipeline
---

### Pipeline Files
* `docker-compose.yml`: Specifies Docker containers and connections required for the pipeline.
* `stream` directory: Stores files used to stream, filter, and write to Hive.
    * `[action].py` files: Sets up the [action]'s Spark streaming job.
    * `template.py`: Template file used to facilitate the setup of Spark jobs.
* `app` directory: stores files that generate random events, dictate business logic, and write to Sqllite tables.
    * `game_api.py`: Stores business logic for various types of actions (discussed below).
    * `models.py`: Writes event data to Sqllite tables.
    * `events.py`: Script for generating random events.

### Pipeline Components
* Flask app: Server that runs the game.
* Kafka topic: There is a single Kafka topic events used in this data pipeline. From this topic, the various event types are filtered and written to Hive tables using separate action-specific Spark streams.
    * Kafka observer creation:
    `docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning`
    
    
* Sqllite tables: The following 5 tables store data generated by the game API for reference when generating random events.
    * swords
    * transactions
    * guilds
    * players
    * guild_interactions
    
    
* Spark jobs: There are action-specific Spark jobs that filter events into their respective Hive tables.


* HDFS Parquet files: The following Parquet files are written by the Spark jobs to their respective Hive directories:
    * `/tmp/add_sword`
    * `/tmp/purchase_sword`
    * `/tmp/add_guild`
    * `/tmp/add_player`
    * `/tmp/join_guild`
    
    
* Hive tables: The following 5 action-specific hive tables are created for querying by Presto:
    * swords
    * sword_transactions
    * guilds
    * players
    * guild_membership


### Game API
---

* We expand the game API to include more types of realistic events and generate additional types of data. The different event types are: "create sword", "create player", "create guild", "join/leave guild", and "purchase sword".
    * We believe this is an appropriate level of complexity for improving our data pipeline design, coding, and querying.
    * Some of these events require additional parameters typical of a multiplayer game.
    * These actions may return additional data on interactions between players, between players and objects, as well as temporal data. This provides us with a rich dataset that allows for more in-depth business queries using Presto.


* We highlight the following noteworthy implications of this added complexity:
    * The Sqllite database allow for simulating realistic in-game events in a persistent game world from within the Flask app, without needing to query the Hive tables.
    * Event data written to the final Hive tables is captured as a string that is then parsed using regex, depending on the analytics question. See the Event Analysis section below.

## Event Generation
---

### From the Command Line
* Events can be generated manually. Depending on the action, there may be required parameters that need to be specified. Examples of how to create each event from the terminal are provided below:
    * Create sword:
    
        `docker-compose exec mids curl "http://localhost:5000/add_sword?cost=100"`
    * Create player:
    
        `docker-compose exec mids curl "http://localhost:5000/add_playter?name=batman?money=999999"`
    * Create guild:
    
        `docker-compose exec mids curl "http://localhost:5000/add_guild?name=jokers"`
    * Join guild:
    
        `docker-compose exec mids curl "http://localhost:5000/join_guild?player_id=1&guild_id=1&join=1"`
    * Leave guild:
    
        `docker-compose exec mids curl "http://localhost:5000/join_guild?player_id=1&guild_id=1&join=0"`
    * Purchase sword:
    
        `docker-compose exec mids curl "http://localhost:5000/purchase_sword?buyer_id=1&sword_id=1"`

### Random Event Generation
- Random game events can be automatically generated using the events.py file:

    `docker-compose exec mids /w205/w205-project3/app/events.py`
    
    - This file is run from `startup.sh` and generates 1000 random events.


* Each event has a random type and resembles the example commands shown above with random parameters.


* Sqllite queries are made to ensure the validity of a randomly generated event.
    * For example, if an event is assigned the "join guild" type, a Sqllite query is made for a random player that does not belong to a guild.

## Event Analysis
---

### Presto Queries

* Once the pipeline has been set up and random events simulating game behavior have been generated, a variety of business questions can be explored using Presto queries.
* Presto can be started from the terminal using the following command:


`docker-compose exec presto presto --server presto:8080 --catalog hive --schema default`


* We provide a few examples of Presto queries that can provide business value:
#### Who are the players?
    
```
SELECT
    distinct regexp_extract(event_body,'(?<=name\": \")(.*?)(?="}})') as distinct_players
FROM
    players
LIMIT 20;
```


Sample results:

| distinct_players |
| ---------------- |
| aFWhMRZAdb       |
| XNExjoSDEn       |
| CtEpgayorA       |
| ahflQgarWJ       |
| AWevpYodeC       |
| ZXcBqruYHd       |
| JgegpOjOlC       |
| hPVMSngaiJ       |
| FjivCeUhkS       |
| zfIWfkijvJ       |
| aUzBuykZcR       |
| wMwTHACbgt       |
| CUeyZUsLFK       |
| rUAlPDGulC       |
| GHzbfDoOjH       |
| JjSTsBprew       |
| cjTBtzgyva       |
| batman           |
| robin            |
| FuFNjhFIWi       |

#### How many guilds are there?

```
SELECT
    count(distinct regexp_extract(event_body,'(?<=name\": \")(.*?)(?="}})') ) as number_of_guilds
FROM
    guilds;
```

Sample Result:

| number_of_guilds |
|------------------|
|              210 |


#### Which sword has been purchased the most?

```
SELECT
    regexp_extract(event_body,'(?<=sword_id\": )(.*?)(?=, \"trans)') as sword_id,
    count( regexp_extract(event_body,'(?<=sword_id\": )(.*?)(?=, \"trans)') ) as count_of_transactions
FROM
    sword_transactions
GROUP BY
    regexp_extract(event_body,'(?<=sword_id\": )(.*?)(?=, \"trans)')
ORDER BY
    count( regexp_extract(event_body,'(?<=sword_id\": )(.*?)(?=, \"trans)') ) DESC
LIMIT 15;
```

Sample Result:

|sword_id | count_of_transactions |
|---------|-----------------------|
|1        |                   114 |
| 6       |                    80 |


#### Which guild and player have the longest membership?

Note that for the query below, any result of '9999' indicates that the player is still a member of the guild.


```
SELECT d.guild_id, d.player_id,
    CASE
        WHEN d.membership_length > 500 THEN 9999
        ELSE d.membership_length
        END as duration
FROM(
    SELECT c.guild_id, c.player_id, sum(duration) as membership_length
    FROM (
        SELECT b.guild_id, b.player_id,
            CASE
                WHEN b.action = 'true' THEN date_diff('minute', b.timestamp, current_timestamp)
                WHEN b.action = 'false' THEN -date_diff('minute', b.timestamp, current_timestamp)
                END as duration
        FROM(
            SELECT *
            FROM (
                SELECT * , rank() over (
                    PARTITION BY a.guild_id, a.player_id
                    ORDER BY a.timestamp DESC) as rank
                FROM (
                    SELECT
                        regexp_extract(event_body,'(?<=guild_id\": )(.*?)(?=}})') as guild_id,
                        regexp_extract(event_body,'(?<=player_id\": )(.*?)(?=, \"timestamp)') as player_id,
                        regexp_extract(event_body,'(?<=join\": )(.*?)(?=, \"player)') as action,
                        date_add('year', 100, date_parse(regexp_extract(event_body,'(?<=timestamp\": \")(.*?)(?=\", \"guild)'), '%m/%d/%Y %T')) as timestamp
                    FROM
                        guild_membership) a )
            WHERE rank = 1 OR rank = 2 ) b ) c
GROUP BY
    c.guild_id, c.player_id) d
ORDER BY
    duration DESC
LIMIT 10;
```


Sample Result:

| guild_id | player_id | duration |
|----------|-----------|----------|
| 33       | 74        |     9999 |
| 24       | 100       |     9999 |
| 176      | 84        |     9999 |
| 45       | 179       |     9999 |
| 90       | 109       |        3 |
| 1        | 1         |        2 |
| 11       | 1         |        2 |
| 21       | 31        |        2 |
| 4        | 10        |        2 |
| 162      | 97        |        1 |


### Notes

* The query logic (especially in dealing with transaction data) may not robustly account for faulty business logic. We believe the proper place to enforce business logic is further upstream in the application pipeline, outside the scope of this project.


* Because of how the event data is captured, regex is used to isolate the parameters of interest. We felt that it was beyond the scope of this project to create custom tables to service every type of business question that we could potentially explore.


* We will run through some of these queries during our live demo to demonstrate how they are used.